In [2]:
from astropy.coordinates import SkyCoord
import numpy as np
from IntegralQuery import SearchQuery, IntegralQuery, Filter, Range
from IntegralPointingClustering import ClusteredQuery
import astropy.io.fits as fits
from astropy.table import Table
from datetime import datetime
import matplotlib.pyplot as plt
import math
from numba import njit
from pyspi.utils.function_utils import find_response_version
from pyspi.utils.response.spi_response_data import ResponseDataRMF
from pyspi.utils.response.spi_response import ResponseRMFGenerator
from pyspi.utils.response.spi_drm import SPIDRM
from pyspi.utils.livedets import get_live_dets
from astromodels import Powerlaw, Log_uniform_prior, Uniform_prior, PointSource, SpectralComponent
from chainconsumer import ChainConsumer
import pymultinest
import os
import astropy.time as at
from scipy.stats import poisson
import pickle

[WARNING ] The naima package is not available. Models that depend on it will not be available
[WARNING ] The GSL library or the pygsl wrapper cannot be loaded. Models that depend on it will not be available.
[WARNING ] The ebltable package is not available. Models that depend on it will not be available
[INFO    ] Starting 3ML!
[WARNING ] no display variable set. using backend for graphics without display (agg)
[WARNING ] ROOT minimizer not available
[WARNING ] PyGMO is not available
[WARNING ] The cthreeML package is not installed. You will not be able to use plugins which require the C/C++ interface (currently HAWC)
[WARNING ] Could not import plugin HAWCLike.py. Do you have the relative instrument software installed and configured?
[WARNING ] Could not import plugin FermiLATLike.py. Do you have the relative instrument software installed and configured?
[WARNING ] No fermitools installed
[WARNING ] Env. variable OMP_NUM_THREADS is not set. Please set it to 1 for optimal performances 

In [3]:
def extract_date_range(path):
    with fits.open(f"{path}/pointing.fits") as file:
        t = Table.read(file[1])
        t1 = at.Time(f'{t["TSTART"][0]+2451544.5}', format='jd')
        t1.format = "isot"
        t2 = at.Time(f'{t["TSTOP"][-1]+2451544.5}', format='jd')
        t2.format = "isot"
    return t1.value, t2.value

def get_scw_ids(path, radius_around_crab, min_time_elapsed, print_results=False):
    p = SkyCoord(83.6333, 22.0144, frame="icrs", unit="deg")
    searchquerry = SearchQuery(position=p, radius=f"{radius_around_crab} degree")
    cat = IntegralQuery(searchquerry)

    f = Filter(SCW_TYPE="POINTING",
               TIME=Range(*extract_date_range(path)))
    scw_ids_all = cat.apply_filter(f, return_coordinates=True, remove_duplicates=True)
    
    scw_ids = []
    
    multiple_files = []
    no_files = []
    no_pyspi = []
    
    num_dets = 19
    eb = np.geomspace(18, 2000, 5)
    emod = np.geomspace(18, 2000, 5)
    for scw_id in scw_ids_all:
        good = True
        with fits.open(f"{path}/pointing.fits") as file:
            t = Table.read(file[1])
            index = np.argwhere(t["PTID_ISOC"]==scw_id[0][:8])
            
            if len(index) < 1:
                no_files.append(scw_id)
                good = False
                continue
                
            elif len(index) > 1:
                multiple_files.append(scw_id)
                good = False
                            
            pointing_info = t[index[-1][0]]
        
            t1 = at.Time(f'{pointing_info["TSTART"]+2451544.5}', format='jd').datetime
            time_start = datetime.strftime(t1,'%y%m%d %H%M%S')
                            
            with fits.open(f"{path}/dead_time.fits") as file2:
                t2 = Table.read(file2[1])
                
                time_elapsed = np.zeros(num_dets)
                
                for i in range(num_dets):
                    for j in index:
                        time_elapsed[i] += t2["LIVETIME"][j[0]*85 + i]
                            
            dets = get_live_dets(time=time_start, event_types=["single"])
                            
            if not np.amin(time_elapsed[dets]) > min_time_elapsed:
                good = False
        
        try: # investigate why this is necessary
            version1 = find_response_version(time_start)
            rsp_base = ResponseDataRMF.from_version(version1)
            rsp1 = ResponseRMFGenerator.from_time(time_start, dets[0], eb, emod, rsp_base)
        except:
            no_pyspi.append(scw_id)
            good = False
            
        if good:
            scw_ids.append(scw_id)
            
    if print_results:
        print("Multiple Files:")
        print(multiple_files)
        print("No Files:")
        print(no_files)
        print("No PySpi:")
        print(no_pyspi)
        print("Good:")
        print(scw_ids)
    
    return np.array(scw_ids)

def create_pair_clusters_crab(
    paths,
    min_angle_dif,
    max_angle_dif,
    max_time_dif,
    radius_around_crab,
    min_time_elapsed
): # check if dets is same in pair?
    output = []
    for path in paths:
        scw_ids = get_scw_ids(path, radius_around_crab, min_time_elapsed)
        cq = ClusteredQuery(scw_ids,
                            angle_weight=0.,
                            time_weight=1./max_time_dif,
                            max_distance=1.,
                            min_ang_distance=min_angle_dif,
                            max_ang_distance=max_angle_dif,
                            cluster_size_range = (2,2),
                            failed_improvements_max = 3,
                            suboptimal_cluster_size = 1,
                            close_suboptimal_cluster_size = 1
                            ).get_clustered_scw_ids()
        for pair in cq[2]:
            output.append((path, pair[0], pair[1]))
    return tuple(output)

In [4]:
def save_pair_clusters(pointings, folder):
    if not os.path.exists(f"./{folder}"):
        os.mkdir(folder)
    with open(f"./{folder}/pointings", "wb") as f:
        pickle.dump(pointings, f)
        
def load_pair_clusters(folder):
    with open(f"./{folder}/pointings", "rb") as f:
        pointings = pickle.load(f)
    return pointings

In [5]:
def extract_pointing_info(path, p_id):
    num_dets = 19
    with fits.open(f"{path}/pointing.fits") as file:
        t = Table.read(file[1])
        index = np.argwhere(t["PTID_ISOC"]==p_id[:8])
        
        if len(index) < 1:
            raise Exception(f"{p_id} not found")

        pointing_info = t[index[-1][0]]
        
        t1 = at.Time(f'{pointing_info["TSTART"]+2451544.5}', format='jd').datetime
        time_start = datetime.strftime(t1,'%y%m%d %H%M%S')
            
    with fits.open(f"{path}/dead_time.fits") as file:
        t = Table.read(file[1])
        
        time_elapsed = np.zeros(num_dets)
        
        for i in range(num_dets):
            for j in index:
                time_elapsed[i] += t["LIVETIME"][j[0]*85 + i]
        
    with fits.open(f"{path}/energy_boundaries.fits") as file:
        t = Table.read(file[1])
        
        energy_bins = np.append(t["E_MIN"], t["E_MAX"][-1])
    
    with fits.open(f"{path}/evts_det_spec.fits") as file:
        t = Table.read(file[1])
        
        counts = np.zeros((num_dets, len(energy_bins)-1))
        for i in range(num_dets):
            for j in index:
                counts[i, : ] += t["COUNTS"][j[0]*85 + i]

    return time_start, time_elapsed, energy_bins, counts

In [6]:
def prepare_fit_data(
    pointings,
    binning_func,
    energy_range
):
    ebs = [] # do i need this?
    counts = []
    dets = []
    t_elapsed = []
    t_start = []
        
    for pair in pointings:
        time_start1, time_elapsed1, energy_bins1, counts_f1 = extract_pointing_info(pair[0], pair[1])
        time_start2, time_elapsed2, energy_bins2, counts_f2 = extract_pointing_info(pair[0], pair[2])
        t_start.append((time_start1, time_start2))
        
        version1 = find_response_version(time_start1)
        version2 = find_response_version(time_start2)
        assert version1 == version2, f"Response versions are not equal for {pair[1]} and {pair[2]}"
        
        dets1 = get_live_dets(time=time_start1, event_types=["single"])
        dets2 = get_live_dets(time=time_start2, event_types=["single"])
        assert np.array_equal(dets1, dets2), f"Active detectors are not the same for {pair[1]} and {pair[2]}"
        dets.append(dets1)
        
        t_elapsed.append((time_elapsed1[dets1], time_elapsed2[dets1]))
        
        assert np.array_equal(energy_bins1, energy_bins2), f"Energy bins are not the same for {pair[1]} and {pair[2]}"
        
        # if max_energy:
        #     for i, e in enumerate(energy_bins1):
        #         if e > max_energy:
        #             energy_bins1 = energy_bins1[:i]
        #             counts_f1 = counts_f1[:i]
        #             counts_f2 = counts_f2[:i]
        #             break
        
        # eb, c = rebin_data_full(energy_bins1,
        #                         np.append(counts_f1[dets1], counts_f2[dets1], axis=0),
        #                         min_counts,
        #                         max_num_bins)
        # counts.append((c[:len(dets1)], c[len(dets1):]))
    
    ebs = tuple(ebs) 
    counts = tuple(counts)
    dets = tuple(dets)
    t_elapsed = tuple(t_elapsed)
    t_start = tuple(t_start)
    
    return ebs, counts, dets, t_elapsed, t_start

In [7]:
def multinest_for_pointing_pairs(
    pointings,
    sources,
    parameter_names,
    true_values_main,
    binning_func,
    energy_range,
    emod,
):
    
    pass
    
    

In [10]:
def rebin_data_exp(
    bins,
    counts,
    energy_range
):

    if energy_range[0]:
        for i, e in enumerate(bins):
            if e > energy_range[0]:
                bins = bins[i:]
                counts = counts[i:]
                break
    if energy_range[1]:
        for i, e in enumerate(bins):
            if e > energy_range[1]:
                bins = bins[:i]
                counts = counts[:i]
                break
    
    min_counts = 5
    
    max_num_bins = 120
    min_num_bins = 1
    
    finished = False
    
    while not finished:
        num_bins = round((max_num_bins + min_num_bins) / 2)
        
        if num_bins == max_num_bins or num_bins == min_num_bins:
            num_bins = min_num_bins
            finished = True
        
        temp_bins = np.geomspace(bins[0], bins[-1], num_bins+1)
        
        new_bins, new_counts = rebin_closest(bins, counts, temp_bins)
        
        if np.amin(new_counts) < min_counts:
            max_num_bins = num_bins
        else:
            min_num_bins = num_bins
            
    return new_bins, new_counts
    
    
def rebin_closest(bins, counts, temp_bins):
    bin = len(bins) - 1
    for i, e in reversed(list(enumerate(temp_bins))):
        for j in range(bin, -1, -1):
            pass
            
            
counts = np.linspace(1,40,40).reshape((2,20))
bins = np.linspace(1,21,21)
temp_bins = np.geomspace(1,21,11)
rebin_closest(bins, counts, temp_bins)

10 21.0 21
10 21.0 20
10 21.0 19
10 21.0 18
10 21.0 17
10 21.0 16
10 21.0 15
10 21.0 14
10 21.0 13
10 21.0 12
10 21.0 11
10 21.0 10
10 21.0 9
10 21.0 8
10 21.0 7
10 21.0 6
10 21.0 5
10 21.0 4
10 21.0 3
10 21.0 2
10 21.0 1
10 21.0 0
9 15.488072227168733 21
9 15.488072227168733 20
9 15.488072227168733 19
9 15.488072227168733 18
9 15.488072227168733 17
9 15.488072227168733 16
9 15.488072227168733 15
9 15.488072227168733 14
9 15.488072227168733 13
9 15.488072227168733 12
9 15.488072227168733 11
9 15.488072227168733 10
9 15.488072227168733 9
9 15.488072227168733 8
9 15.488072227168733 7
9 15.488072227168733 6
9 15.488072227168733 5
9 15.488072227168733 4
9 15.488072227168733 3
9 15.488072227168733 2
9 15.488072227168733 1
9 15.488072227168733 0
8 11.422875300666448 21
8 11.422875300666448 20
8 11.422875300666448 19
8 11.422875300666448 18
8 11.422875300666448 17
8 11.422875300666448 16
8 11.422875300666448 15
8 11.422875300666448 14
8 11.422875300666448 13
8 11.422875300666448 12
8 11.42287

In [8]:
def crab_sources():
    sources = []
    parameter_names = []
    
    piv = 40
    ra_crab, dec_crab = 83.6333, 22.0144
    crab_parameters = np.array([[9.3, 1, -2.08],
                                [7.52e-4, 100, -1.99],
                                [11.03, 1, -2.1]])
    
    true_values_main = np.zeros((len(crab_parameters),2))
    true_values_main[:,0] = crab_parameters[:,0] * (piv / crab_parameters[:,1])**crab_parameters[:,2]
    true_values_main[:,1] = crab_parameters[:,2]

    crab_pl = Powerlaw()
    crab_pl.index = true_values_main[0,1]
    crab_pl.K = true_values_main[0,0]
    crab_pl.piv = piv
    crab_pl.K.prior = Log_uniform_prior(lower_bound=1e-6, upper_bound=1e-0)
    crab_pl.index.prior = Uniform_prior(lower_bound=-4, upper_bound=0)
    crab_component1 = SpectralComponent('crab_pl',shape=crab_pl)
    crab_ps = PointSource('crab_plline',ra=ra_crab, dec=dec_crab, components=[crab_component1])
    
    sources.append((crab_pl, crab_ps))
    parameter_names.extend(["Crab K", "Crab index"])
    
    ra_1A, dec_1A = 84.7270, 26.3160
    _1A_pl = Powerlaw()
    # _1A_pl.index = -2
    # _1A_pl.K = 0.005
    _1A_pl.piv = 40
    _1A_pl.K.prior = Log_uniform_prior(lower_bound=1e-7, upper_bound=1e1)
    _1A_pl.index.prior = Uniform_prior(lower_bound=-10, upper_bound=4)
    _1A_component1 = SpectralComponent('_1A_pl',shape=_1A_pl)
    _1A_ps = PointSource('_1A_plline',ra=ra_1A, dec=dec_1A, components=[_1A_component1])
    
    sources.append((_1A_pl, _1A_ps))
    parameter_names.extend(["1A K", "1A index"])
    
    return tuple(sources), parameter_names, true_values_main


In [9]:
folder = "orbit_1019"

# save_pair_clusters(
#     create_pair_clusters_crab(
#         ("crab_data/1019",),
#         min_angle_dif=1.5,
#         max_angle_dif=4.,
#         max_time_dif=0.2,
#         radius_around_crab=5.,
#         min_time_elapsed=600.
#     ),
#     folder
# )

pointings = load_pair_clusters(folder)
sources, parameter_names, true_values_main = crab_sources()

